#### Conectando Python com API do Google Sheets

In [22]:
import pandas as pd
import gspread
import json
import os
import warnings
from dotenv import load_dotenv

warnings.filterwarnings("ignore")

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()
key_json = os.getenv('KEY_JSON')

# Converte a string JSON em um dicionário Python
google_client = gspread.service_account_from_dict(json.loads(key_json))

# Identificador único da planilha do Google Sheets
SHEET_ID = '1tbJh_GQol0Ax5uB-AjwI26yW9piCIp0peuKV1N5nMeQ'

# Abre a planilha do Google Sheets usando o código único da planilha
spreadsheet = google_client.open_by_key(SHEET_ID)

# Seleciona a aba chamada 'Dados' dentro da planilha
worksheet = spreadsheet.worksheet('Dados')

# Obtém todos os valores da worksheet (retorna uma lista de listas)
dados = worksheet.get_all_values()

# A primeira linha é usada como o cabeçalho (nomes das colunas)
df = pd.DataFrame(dados[1:], columns=dados[0])

#### Limpeza e Preparação dos dados

In [23]:
# Padronizando o nome das colunas
df.rename(columns=str.lower, inplace=True)

# Alterando o tipo de dado do DataFrame
colunas = ['tempo_espera', 'tempo_expedicao', 'tempo_total']
df[colunas] = df[colunas].apply(pd.to_timedelta)
df['qtd_remessas'] = df['qtd_remessas'].astype('int')

In [27]:
 # Tempo Médio de Expedição por Data
tempo_medio_dia = df.groupby(['data', 'unidade']).agg(
                    media_tempo_espera = pd.NamedAgg('tempo_espera', 'mean'),
                    media_tempo_expedicao = pd.NamedAgg('tempo_expedicao', 'mean'),
                    media_tempo_total = pd.NamedAgg('tempo_total', 'mean'),
                    total_remessas = pd.NamedAgg('qtd_remessas', 'sum')).reset_index()

def format_timedelta(td):
    # Formata como HH:MM:SS
    total_seconds = int(td.total_seconds())
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{hours:02}:{minutes:02}:{seconds:02}"

# Aplicar a formatação das médias
tempo_medio_dia[['media_tempo_espera',
                 'media_tempo_expedicao',
                 'media_tempo_total']] = tempo_medio_dia[['media_tempo_espera',
                                                         'media_tempo_expedicao',
                                                         'media_tempo_total']].applymap(format_timedelta)

tempo_medio_dia

,data,unidade,media_tempo_espera,media_tempo_expedicao,media_tempo_total,total_remessas
0,15/10/2024,HVGI - VILA GUILHERME,00:56:54,00:27:35,01:24:29,346
1,16/10/2024,HVGI - VILA GUILHERME,00:29:43,00:16:24,00:46:07,152
2,18/10/2024,HVGI - VILA GUILHERME,01:52:03,00:48:34,02:40:38,4277


In [28]:
tempo_medio_dia.to_clipboard(index=False)